# Project 1 (Due Nov 13)

The goal of the first project is to non-parametrically model some phenomenon of interest, and generate sequences of values. There are six options below:

- Chordonomicon: 680,000 chord progressions of popular music songs. Create a chord generator, similar to what we did with Bach in class, but for a particular artist or genre. (https://github.com/spyroskantarelis/chordonomicon)
- Financial Time series, S&P500 Stocks: There are 500 time series here. Model how individual time series adjust over time, either together or separately. (https://www.kaggle.com/datasets/andrewmvd/sp-500-stocks)
- MIT-BIT Arrythmia Database: Arrythmia is an abnormal heart rhythm. This is a classic dataset that a day of ECG time series measurements for 4,000 patients. (https://www.physionet.org/content/mitdb/1.0.0/)
- Ukraine conflict monitor: The ACLED Ukraine Conflict Monitor provides near real-time information on the ongoing war in Ukraine, including an interactive map, a curated data file, and weekly situation updates Ukraine Conflict Monitor, maintained by the Armed Conflict Location & Event Data Project, starting in 2022, including battles, explosions/remote violence, violence against civilians, protests, and riots:
https://acleddata.com/monitor/ukraine-conflict-monitor
- SIPRI Arms Trade: The SIPRI Arms Transfers Database is a comprehensive public resource tracking all international transfers of major conventional arms from 1950 to the present. For each deal, information includes: number ordered, supplier/recipient identities, weapon types, delivery dates, and deal comments. The database can address questions about: who are suppliers and recipients of major weapons, what weapons have been transferred by specific countries, and how supplier-recipient relationships have changed over time.
https://www.sipri.org/databases/armstransfers
- Environmental Protection Agency data: The EPA, in general, has excellent data on the release of toxic substances, and I also tracked down air quality and asthma. You can put these together to look at how changes in toxic release correlate with air quality and respiratory disease over time:
https://www.epa.gov/data
https://www.epa.gov/toxics-release-inventory-tri-program/tri-toolbox
https://www.cdc.gov/asthma/most_recent_national_asthma_data.htm
https://www.earthdata.nasa.gov/topics/atmosphere/air-quality/data-access-tools

If you have other data sources that you're interested in, I am willing to consider them, as long as they lend themselves to an interesting analysis.

Submit a document or notebook that clearly addresses the following:
1. Describe the data clearly -- particularly any missing data that might impact your analysis -- and the provenance of your dataset. Who collected the data and why? (10/100 pts)
2. What phenomenon are you modeling? Provide a brief background on the topic, including definitions and details that are relevant to your analysis. Clearly describe its main features, and support those claims with data where appropriate. (10/100 pts)
3. Describe your non-parametric model (empirical cumulative distribution functions, kernel density function, local constant least squares regression, Markov transition models). How are you fitting your model to the phenomenon to get realistic properties of the data? What challenges did you have to overcome? (15/100 pts)
4. Either use your model to create new sequences (if the model is more generative) or bootstrap a quantity of interest (if the model is more inferential). (15/100 pts)
5. Critically evaluate your work in part 4. Do your sequences have the properties of the training data, and if not, why not? Are your estimates credible and reliable, or is there substantial uncertainty in your results? (15/100 pts)
6. Write a conclusion that explains the limitations of your analysis and potential for future work on this topic. (10/100 pts)

In addition, submit a GitHub repo containing your code and a description of how to obtain the original data from the source. Make sure the code is commented, where appropriate. Include a .gitignore file. We will look at your commit history briefly to determine whether everyone in the group contributed. (10/100 pts)

In class, we'll briefly do presentations and criticize each other's work, and participation in your group's presentation and constructively critiquing the other groups' presentations accounts for the remaining 15/100 pts.


## Data Description and Provenance

In [1]:
import pandas as pd
import numpy as np


# Load files
companies = pd.read_csv('data/sp500_companies.csv')
index_df  = pd.read_csv('data/sp500_index.csv')
stocks    = pd.read_csv('data/sp500_stocks.csv')


# Convert dates
index_df['Date'] = pd.to_datetime(index_df['Date'], errors='coerce')
stocks['Date']   = pd.to_datetime(stocks['Date'],   errors='coerce')


def missing_report(df, name):
   print(f"\n=== {name} ===")
   print(f"Shape: {df.shape}")
   if 'Date' in df.columns:
       print(f"Date range: {df['Date'].min()} to {df['Date'].max()}")
   missing = df.isnull().sum()
   percent = 100 * missing / len(df)
   report = pd.DataFrame({'Missing Count': missing, 'Percent Missing': percent})
   report = report[report['Missing Count'] > 0].sort_values('Percent Missing', ascending=False)
   if not report.empty:
       print("Missing values:")
       print(report.round(3))
   else:
       print("No missing values in any column.")
   print("-" * 50)


missing_report(companies, "data/sp500_companies.csv")
missing_report(index_df,  "data/sp500_index.csv")
missing_report(stocks,    "data/sp500_stocks.csv")


=== data/sp500_companies.csv ===
Shape: (502, 16)
Missing values:
                   Missing Count  Percent Missing
Ebitda                        29            5.777
State                         20            3.984
Fulltimeemployees              9            1.793
Revenuegrowth                  3            0.598
--------------------------------------------------

=== data/sp500_index.csv ===
Shape: (2517, 2)
Date range: 2014-12-22 00:00:00 to 2024-12-20 00:00:00
No missing values in any column.
--------------------------------------------------

=== data/sp500_stocks.csv ===
Shape: (1891536, 8)
Date range: 2010-01-04 00:00:00 to 2024-12-20 00:00:00
Missing values:
           Missing Count  Percent Missing
Adj Close         100475            5.312
Close             100475            5.312
High              100475            5.312
Low               100475            5.312
Open              100475            5.312
Volume            100475            5.312
------------------------------

### Data Description and Missing Value Summary

This project uses three related datasets describing the S&P 500 index and its constituent companies.

(a) sp500_companies.csv

- Shape: (502 × 16)  
- Description: Contains company-level metadata and financial indicators for S&P 500 constituents.  
- Key columns: Exchange, Symbol, Shortname, Sector, Industry, Currentprice, Marketcap, Ebitda, Revenuegrowth, Fulltimeemployees, etc.  

Missing Data:

| Column             | Missing Count | Percent Missing |
|--------------------|---------------|-----------------|
| Ebitda             | 29            | 5.78%           |
| State              | 20            | 3.98%           |
| Fulltimeemployees  | 9             | 1.79%           |
| Revenuegrowth      | 3             | 0.60%           |

Interpretation: 
Most missing data appear in financial fields (Ebitda, Revenuegrowth) and location-related fields (State).  
These gaps likely stem from incomplete corporate disclosures or companies headquartered outside the U.S.  
The dataset otherwise has complete coverage of essential identification and classification fields such as Symbol and Sector.

(b) sp500_index.csv

- Shape: (2,517 × 2)  
- Date range: 2014-12-22 → 2024-12-20  
- Columns: Date, S&P500 (daily closing level).  
- Missing data: None detected.  

Interpretation:  
The index file is fully complete, covering approximately ten years of daily observations.  
This makes it an ideal foundation for modeling index-level returns or volatility.

(c) sp500_stocks.csv

- Shape: (1,891,536 × 8)  
- Date range: 2010-01-04 → 2024-12-20  
- Columns: Date, Symbol, Open, High, Low, Close, Adj Close, Volume.  

Missing Data:

| Column     | Missing Count | Percent Missing |
|-------------|----------------|-----------------|
| Adj Close   | 100,475        | 5.31%           |
| Close       | 100,475        | 5.31%           |
| High        | 100,475        | 5.31%           |
| Low         | 100,475        | 5.31%           |
| Open        | 100,475        | 5.31%           |
| Volume      | 100,475        | 5.31%           |

Interpretation:  
We will investigate the missing values further in the next code block.

### Data Provenance

The datasets used in this project originate from the Kaggle dataset  
**[“S&P 500 Stocks”](https://www.kaggle.com/datasets/andrewmvd/sp-500-stocks)**.  
This dataset aggregates information about S&P 500 companies, their historical prices, and the overall index level.

Source Details:
- Platform: Kaggle  
- Dataset Owner: Larxel (dataset maintainer)  
- Collection Methodology: Data collected from the Federal Reserve Economic Data (FRED) and the Yahoo Finance (yfinance) API.  
- License: CC0: Public Domain 
- Update Frequency: Daily (last updated approximately one year ago)  
- Temporal Coverage: 2010–2024   
- Dataset Version Used: v1022 (the most recent reliable version)  

> Note:
> The latest version of the dataset (v1023) contains approximately 67% missing values, rendering it unsuitable for analysis.  
> Therefore, this project uses version 1022, which provides complete and consistent coverage for the S&P 500 index and its constituent stock data. 

In [4]:
price_cols = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
full_missing = stocks[price_cols].isna().all(axis=1)


# 1. Count missing rows per ticker
missing_per_ticker = (
   full_missing.groupby(stocks['Symbol'])
   .sum()
   .astype(int)
   .sort_values(ascending=False)
)


# 2. Show results
print(f"Total missing rows: {full_missing.sum():,}")
print(f"Tickers with missing data: {missing_per_ticker[missing_per_ticker > 0].shape[0]}")


print("\n--- Top 15 tickers with MOST missing days ---")
print(missing_per_ticker.head(15).to_string())

Total missing rows: 100,475
Tickers with missing data: 72

--- Top 15 tickers with MOST missing days ---
Symbol
AMTM    3705
SW      3650
GEV     3583
SOLV    3583
VLTO    3461
KVUE    3356
GEHC    3261
CEG     3032
ABNB    2754
PLTR    2704
OTIS    2569
CARR    2569
CRWD    2375
CTVA    2363
UBER    2353


### Missing values in `sp500_stocks.csv'

We examined which specific tickers had missing data in the sp500_stocks.csv dataset.  
Out of approximately 1.89 million rows (covering 2010–2024), there are 100,475 rows with missing values across *all* price and volume columns — about 5.31% of the dataset.  

Only 72 tickers are affected, primarily recent S&P 500 additions such as AMTM (3,705 missing days), SW, GEV, and SOLV (2024 spin-offs), followed by VLTO, KVUE, GEHC, CEG, ABNB, PLTR, OTIS, CARR, CRWD, CTVA, and UBER (added between 2019–2023).  

These gaps correspond to periods before each company joined the index, not data errors.  
They occur because the dataset spans the entire 2010–2024 period, even though many of these firms entered the S&P 500 mid-period.  
The remaining ~430 tickers have complete trading histories.  

This pattern of missingness is structural and expected, reflecting real-world index composition changes rather than issues with data collection.
